In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, date
import time
import csv

In [ ]:
# sanity check get total number of fatalities
df = pd.read_csv('../FARS2018NationalCSV/Person.CSV', encoding="ISO-8859-1")
df = df.loc[df['INJ_SEV'] == 4]
print(len(df))
df.head(5)
df = pd.read_csv('../FARS2018NationalCSV/accident.CSV', encoding="ISO-8859-1")
print(len(df))



# get fatalities of only when there are multiple people in vehicle
df1 = pd.read_csv('../FARS2018NationalCSV/vehicle.csv', encoding="ISO-8859-1")
print(len(df1))
df1 = df1.loc[df1['NUMOCCS'] > 1]
print(len(df1))
df2 = pd.read_csv('../FARS2018NationalCSV/Person.CSV', encoding="ISO-8859-1")
df2 = df2[df2['ST_CASE'].isin(df1['ST_CASE'].values.tolist())]
df = df2.loc[df2['INJ_SEV'] == 4]
print(df['INJ_SEV'][0:20])
print(df['DOA'][0:20]) # dead on arrival
print(len(df))



### Create Dataframe

Parses the CSV files and creates a dataframe with all the fatalities

In [27]:
# manually get all the events
def create_dataframe():
    with open('../FARS2018NationalCSV/vehicle.csv', mode ='r', encoding='mac_roman') as v_file, open('../FARS2018NationalCSV/person.csv', mode='r', encoding='mac_roman') as p_file:
        csvFile = csv.reader(v_file)
        state_cases = []
        vehicle_cases = []
        v_labels = None
        for line in csvFile:
            if v_labels == None:
                v_labels = line
                continue
            num_occs = int(line[5].strip())
            num_deaths = int(line[188].strip())
            if num_occs > 1 and num_occs < 98 and num_deaths > 0:
                # add to list
                state_cases.append(line[2])
                vehicle_cases.append(line)
        print(len(state_cases))
        csvFile = csv.reader(p_file)
        people_list = []
        label_list = None
        for line in csvFile:
            if label_list == None:
                label_list = line
                continue
            # check statecase and then check if they died
            if line[2] in state_cases and int(line[52]) == 4 and (int(line[50]) == 1 or int(line[50]) == 2):
                people_list.append(line)

    # read into a dataframe
    return pd.DataFrame(people_list, columns=label_list), pd.DataFrame(vehicle_cases, columns=v_labels)

In [ ]:
# get fatalities of only when there are multiple people in vehicle
df1 = pd.read_csv('../FARS2018NationalCSV/vehicle.csv', encoding="ISO-8859-1")
print(len(df1))
df1 = df1.loc[(df1['NUMOCCS'] > 1) & (df1['DEATHS'] > 0)]
print(len(df1['ST_CASE']))
df2 = pd.read_csv('../FARS2018NationalCSV/Person.CSV', encoding="ISO-8859-1")
df2 = df2[df2['ST_CASE'].isin(df1['ST_CASE'].values.tolist())]
df = df2.loc[df2['INJ_SEV'] == 4]
print(df['INJ_SEV'][0:20])
print(df['DOA'][0:20]) # dead on arrival
print(len(df))
print(df['ST_CASE'][0:30])

print(df1['ST_CASE'][0:30])

## Graphs about the dataset
#### Graphs about time of crashes and time of death

In [ ]:
# create graph to display time of day crash occurred
df_hour = df['HOUR'].astype('int')
df_hour = df_hour.loc[df_hour <= 24] # dropout unknown crash times to get cleaner histogram

df_unknown_hour = df['HOUR'].astype('int')
df_unknown_hour = df_unknown_hour.loc[df_unknown_hour > 24]

fig,ax = plt.subplots(1,2, figsize=(10,5))
ax[0].hist(df_hour, range(0,25),edgecolor = "black")
_,bins,_ = ax[1].hist(df_unknown_hour, edgecolor="black")
ax[1].set_xticks(bins);

In [ ]:
# graph of time of death -- including deaths on arrival
df_hour = df['DEATH_HR'].astype('int')
df_hour = df_hour.loc[df_hour <= 24] # dropout unknown crash times to get cleaner histogram

df_unknown_hour = df['DEATH_HR'].astype('int')
df_unknown_hour = df_unknown_hour.loc[df_unknown_hour > 24]

fig,ax = plt.subplots(1,2, figsize=(10,5))
_,bins,_ = ax[0].hist(df_hour, range(0,25),edgecolor = "black")
print(bins)
_,bins,_ = ax[1].hist(df_unknown_hour, edgecolor="black")
ax[1].set_xticks(bins);

In [ ]:
# graph of time of death -- excluding deaths on arrival
df_hour = df.loc[df['DOA'].astype('int') != 7]
df_hour = df_hour['DEATH_HR'].astype('int')
df_hour = df_hour.loc[df_hour <= 24] # dropout unknown crash times to get cleaner histogram

df_unknown_hour = df.loc[df['DOA'].astype('int') != 7]
df_unknown_hour = df['DEATH_HR'].astype('int')
df_unknown_hour = df_unknown_hour.loc[df_unknown_hour > 24]

fig,ax = plt.subplots(1,2, figsize=(10,5))
_,bins,_ = ax[0].hist(df_hour, range(0,25),edgecolor = "black")
print(bins)
_,bins,_ = ax[1].hist(df_unknown_hour, edgecolor="black")
ax[1].set_xticks(bins);

In [ ]:
# graph of Dead on Arrival
arr = plt.hist(df['DOA'].astype('int'), range(0,11),edgecolor="black")
arr = arr[0]
print('Note:\n0 -- made it to hospital', arr[0], '\n7 -- dead on arrival', arr[7], '\n8 -- died in transit', arr[8], '\n9 -- Unknown', arr[9])

#### ejection stats

In [ ]:
# 
df_ejected = df['EJECTION'].astype('int')
_,bins,_ = plt.hist(df_ejected, range(0,11),edgecolor = "black")
print(bins)

### Print out some statistics about the dataset

Look at individual statistics and see what can be used.

> Need to narrow down some of the variables

##### somethings to possible look at
- the time of survival for ejections
- how likely is someone to survive if they are ejected
- heatmap that shows DOA, ejections, rollovers

In [ ]:
# cell is used to calculate survival time of all occurrances
# structure of time MMDDHHMM
df_crash_times = df.loc[df['HOUR'] != 99] # drop unknown times
df_crash_times = df_crash_times.loc[(df_crash_times['DEATH_HR'] != 99) & (df_crash_times['DEATH_MO'] != 99) & (df_crash_times['DEATH_MN'] != 99)]
# df_crash_times = pd.DataFrame(crash_dict)
df_crash_times['combined'] = "2018 " + df_crash_times['MONTH'].astype(str) + " " + df_crash_times['DAY'].astype(str) + " 2018 " + df_crash_times['DEATH_MO'].astype(str) + " " + df_crash_times['DEATH_DA'].astype(str)# + " " + df_crash_times['HOUR'].astype(str) + " " + df_crash_times['MINUTE'].astype(str)

crash_times = []
day_diff = []
for i in df_crash_times['combined']:
    date_split = i.split()
    day_diff.append((date(2018, int(date_split[4]), int(date_split[5])) - date(2018, int(date_split[1]), int(date_split[2]))).days)
#     crash_times.append(datetime.timestamp(datetime.strptime(i, "%Y %m %d %H %M")))
day_diff = np.array(day_diff)
day_diff = day_diff[day_diff >= 0]

"""
df_crash_times['combined_death'] = "2018 " + df_crash_times['DEATH_MO'].astype(str) + " " + df_crash_times['DEATH_DA'].astype(str) + " " + df_crash_times['DEATH_HR'].astype(str) + " " + df_crash_times['DEATH_MN'].astype(str)
death_times = []
for i in df_crash_times['combined_death']:
    death_times.append(datetime.timestamp(datetime.strptime(i, "%Y %m %d %H %M")))

diff_times = []
for i in range(0, len(death_times)):
    print(death_times[i])
    diff_times.append(date(death_times[i]) - date(crash_times[i]))
"""

plt.hist(day_diff, 30)
plt.title("Survival Time")
print("mean: ", np.mean(day_diff))
print("std: ", np.std(day_diff))
print("var: ", np.var(day_diff))

In [ ]:
# new way to get survival time -- using lag
lag_hrs = df['LAG_HRS'].astype('int') + df['LAG_MINS'].astype('int') / 60
lag_hrs = lag_hrs[lag_hrs < 900] # drop out unknowns
plt.hist(lag_hrs)
plt.title("Survival Time")
print('mean = ', np.mean(lag_hrs))
print('std = ', np.std(lag_hrs))

In [ ]:
# average age 
df['AGE'].loc[df['AGE'] < 125].describe()

In [ ]:
# amount of people who died at the scene after ejection
df_fulleject = df.loc[(df['EJECTION'] == 2) & (df['DOA'] != 9)]
fig,ax = plt.subplots(1,2, figsize=(10,3))
print('Note:\n0 -- made it to hospital\n7 -- dead on arrival\n8 -- died in trasit')
ax[0].hist(df_fulleject['DOA'], range(0,10));
ax[0].title.set_text('Full ejection')
df_fulleject = df.loc[(df['EJECTION'] == 1) & (df['DOA'] != 9)]
ax[1].hist(df_fulleject['DOA'], range(0,10));
ax[1].title.set_text('Partial ejection')

In [ ]:
# display simple stats about collision location and death rate
df_collision = df.loc[(df['IMPACT1'] != 0) & (df['IMPACT1'] != np.NaN) & (df['IMPACT1'] != 99) & (df['IMPACT1'] != 98) & (df['DOA'] != 9)]
print('x-axis:\n0 -- made it to hospital\n7 -- dead on arrival\n8 -- died in trasit\n')
print('\ny-axis:\n')
plt.scatter(df_collision['DOA'],df_collision['IMPACT1'])

## Get severely injured occupants of vehicles but survived

In [ ]:
# get fatalities of only when there are multiple people in vehicle
df1 = pd.read_csv('../FARS2018NationalCSV/vehicle.csv', encoding="ISO-8859-1")
df1 = df1.loc[df1['NUMOCCS'] > 1]
df2 = pd.read_csv('../FARS2018NationalCSV/Person.CSV', encoding="ISO-8859-1")
df2 = df2[df2['ST_CASE'].isin(df1['ST_CASE'].values.tolist())]
df_survived = df2.loc[(df2['INJ_SEV'] == 2) | (df2['INJ_SEV'] == 3) | (df2['INJ_SEV'] == 0) | (df2['INJ_SEV'] == 1)]
print(df_survived['ST_CASE'][0:20])
print(df_survived['INJ_SEV'][0:20])
print(df_survived['DOA'][0:20]) # dead on arrival
print(len(df_survived))

### Data Notes

Things that might be helpful to use as variable

- collision location
- rollover
- ejection
- speed if available
- airbag deployed or not
- seatbelt usage


### Possible Methods

There are a couple approaches that could be taken for survival time. The response variable will be the survival time which can be found in plots above

- split into bins
 - 4 bins of ≈7
 - use 31 bins
 
#### Regression methods

each of these methods will be continuous variable

- neural network -- most likely will use
- straight linear regression

#### Feature selection

- sklearn feature selection
- wrapper method
- filter method

### Explainable ML tools

- shap


In [4]:
def get_dataframe(fn, case_list):
    with open(fn, mode ='r', encoding='mac_roman') as file:
        csvFile = csv.reader(file)
        contents = []
        label_list = None
        for line in csvFile:
            if label_list == None:
                label_list = line
                continue
            if int(line[2]) in case_list:
                contents.append(line)
    # return dataframe
    return pd.DataFrame(contents, columns=label_list)

In [28]:
# get dataframe of people -- takes a second
df_person, df_v = create_dataframe()
st_case = df_person['ST_CASE'].astype('int').to_numpy()
df_acc = get_dataframe('../FARS2018NationalCSV/accident.csv', st_case)

8502


In [ ]:
# combine to make one large dataframe
df_acc[["RUR_URB","MAN_COLL","TYP_INT","NOT_HOUR","NOT_MIN","ARR_HOUR","ARR_MIN","LGT_COND"]]
df_v[["TRAV_SP","UNDERIDE","ROLLOVER","IMPACT1","DEFORMED","VTRAFWAY","VNUM_LAN","VALIGN","VPROFILE","VPAVETYP","VSURCOND","P_CRASH1","P_CRASH3","PCRASH4","TRAV_SP", "ACC_TYPE"]]
df_person[["AGE","SEX","SEAT_POS","REST_USE","AIR_BAG", "EXTRICAT","EJECTION","EJ_PATH","EXTRICAT","DRINKING","DRUGS","HOSPITAL"]]

#### todo

- add in extra files that are missing
- need to convert to minutes instead of hours
 - change notification time and arrivial time

## plan

- equal splitting of testing and training
 - so there are outliers in both
- feature selection
 - use random forest to select features
- train model
 - using multinomial logistic regression
 - use sklearn.linear_model.LogisticRegression

